In [1]:
import os
from dotenv import load_dotenv
from rich import print as rich_print
import logfire
from __future__ import annotations


from dataclasses import dataclass, field
from pydantic import BaseModel
from typing import List, Optional, Literal
from pydantic_ai import Agent, NativeOutput, RunContext

load_dotenv()

LOGFIRE_WRITE_TOKEN = os.getenv("LOGFIRE_WRITE_TOKEN")
logfire.configure(token=LOGFIRE_WRITE_TOKEN)
logfire.instrument_pydantic_ai()
logfire.instrument_httpx()

run_tests = True

In [2]:
import ollama
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider


OLLAMA_MODEL_NAME = "gpt-oss:20b"
OLLAMA_BASE_URL = "http://localhost:11434"
ollama.pull(OLLAMA_MODEL_NAME)
model = OpenAIChatModel(
    model_name=OLLAMA_MODEL_NAME,
    provider=OllamaProvider(base_url=f"{OLLAMA_BASE_URL}/v1", api_key="ollama"),
)

Logfire project URL: ]8;id=632047;https://logfire-us.pydantic.dev/brad-ito/pydantic-ai-demo\https://logfire-us.pydantic.dev/brad-ito/pydantic-ai-demo]8;;\

10:07:31.953 POST 127.0.0.1/api/pull


In [3]:
initial_search_query = "executive leadership government defense jobs"

with open("GEORGE_WASHINGTON_RESUME.md", "r", encoding="utf-8") as f:
    candidate_resume = f.read()

In [4]:
import os
from tavily import AsyncTavilyClient

tavily_client = AsyncTavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [5]:
class SearchResult(BaseModel):
    page_title: str
    page_description: str
    page_content: str
    url: str


class AnalyzedSearchResult(SearchResult):
    job_title: Optional[str] = None
    job_summary: Optional[str] = None
    should_skip: bool
    is_good_fit: bool
    fit_reasoning: Optional[str] = None


class ProspectiveJob(BaseModel):
    job_title: str
    job_summary: str
    url: str
    draft_cover_letter: str


class GraphOutput(BaseModel):
    prospective_jobs: List[ProspectiveJob]


@dataclass
class GraphDeps:
    search_max_results: int = 10
    max_search_iterations: int = 3
    target_num_prospective_jobs: int = 5


@dataclass
class GraphState:
    search_query: str
    candidate_resume: str
    prospective_jobs: List[ProspectiveJob] = field(default_factory=list)
    search_iteration: int = 0
    search_results: List[SearchResult] = field(default_factory=list)

In [6]:
from pydantic_graph.beta import GraphBuilder, StepContext, TypeExpression


gb = GraphBuilder[GraphState, GraphDeps, None, GraphOutput](
    state_type=GraphState, deps_type=GraphDeps, output_type=GraphOutput
)

# Search Prep


In [7]:
@dataclass
class SearchPrepDeps:
    search_query: str
    search_results: List[SearchResult]


class SearchPrepOutput(BaseModel):
    search_query: str
    reasoning: str


search_prep_agent = Agent[SearchPrepDeps, SearchPrepOutput](
    model,
    deps_type=SearchPrepDeps,
    output_type=NativeOutput[SearchPrepOutput](
        SearchPrepOutput,
        name="SearchPrepOutput",
        description="A refined search query for finding job postings",
    ),
    system_prompt="""
    you are an expert in job recruiting and job search.
    When given a search query, you can vary it in creative ways to find more job postings.
    """,
    name="Search Prep Agent",
)


@search_prep_agent.instructions
async def refine_search_instruction(ctx: RunContext[SearchPrepDeps]) -> str:
    """Refines the search query based on recent job postings."""
    deps = ctx.deps
    if deps.search_query:
        logfire.info("refining search query")
        recent_page_titles = [r.page_title for r in deps.search_results[-5:]]
        titles_str = ", ".join(recent_page_titles)
        instruction = f"""
        Based on these recent job posting titles we found: {titles_str}
        Suggest a refined search query to find similar but different job postings. 
        Current query: {deps.search_query}
        Provide a new refined search query that is under 400 characters.
        """
        return instruction
    else:
        return None


@logfire.instrument()
async def _search_prep(search_query: str, search_results: List[SearchResult]) -> str:
    logfire.info(f"refining {search_query=}")
    result = await search_prep_agent.run(
        deps=SearchPrepDeps(search_query=search_query, search_results=search_results)
    )
    output = result.output
    refined_search_query = output.search_query
    logfire.info(f"refined search query to {refined_search_query}")
    return refined_search_query


@gb.step
async def search_prep(ctx: StepContext[GraphState, GraphDeps, None]) -> str:
    """Prepares the search query for the job search."""
    state = ctx.state
    if state.search_iteration > 0:
        state.search_query = await _search_prep(
            state.search_query, state.search_results
        )
    else:
        logfire.info(f"using initial search query {state.search_query}")
    return state.search_query


In [8]:
if run_tests:
    refined_search_query = await _search_prep(initial_search_query, [])
    rich_print(refined_search_query)

10:07:32.942 Calling __main__._search_prep
10:07:32.945   refining search_query='executive leadership government defense jobs'
10:07:32.946   Search Prep Agent run
10:07:32.950     refining search query
10:07:32.951     chat gpt-oss:20b
10:07:33.116       POST localhost/v1/chat/completions
10:08:06.788   refined search query to ("executive leadership" OR "executive director" OR "senior exe...OR procurement OR risk) NOT (entry OR internship OR associate)


("executive leadership" OR "executive director" OR "senior executive" OR "leadership") AND (government OR federal) 
AND (defense OR "national security" OR cybersecurity) AND (policy OR strategy OR program OR procurement OR risk) 
NOT (entry OR internship OR associate)

# Seach Job Postings


In [9]:
JOB_BOARD_DOMAINS = [
    "myworkdayjobs.com",
    "boards.greenhouse.io",
    "jobs.lever.co",
    "icims.com",
    "smartrecruiters.com",
    "workable.com",
    "jobvite.com",
    "bamboohr.com",
    "breezyhr.com",
    "jobs.ashbyhq.com",
    "taleo.net",
    "paylocity.com",
    "adp.com",
    "jazz.co",
]


@logfire.instrument()
async def _search_job_postings(
    search_query: str, search_max_results: int = 5
) -> List[SearchResult]:
    search_response = await tavily_client.search(
        query=search_query,
        auto_paragraphs=True,
        include_raw_content="markdown",
        max_results=search_max_results,
        include_domains=JOB_BOARD_DOMAINS,
        country="united states",
    )

    results = search_response.get("results", [])
    search_results = [
        SearchResult(
            page_title=result.get("title") or "",
            page_description=result.get("description") or "",
            page_content=result.get("raw_content") or "",
            url=result.get("url") or "",
        )
        for result in results
        if result.get("url")
    ]
    return search_results


@gb.step
async def search_job_postings(
    ctx: StepContext[GraphState, GraphDeps, str],
) -> List[SearchResult]:
    """Calls the tavily search tool to find new job postings, and tweaks the search query based on recent job postings."""
    search_query = ctx.inputs
    deps = ctx.deps
    state = ctx.state
    state.search_iteration += 1

    search_results = await _search_job_postings(search_query, deps.search_max_results)

    state.search_results = search_results
    return search_results

In [10]:
if run_tests:
    search_results = await _search_job_postings("AI engineer in Dallas, TX", 5)
    rich_print(search_results)

10:08:06.802 Calling __main__._search_job_postings
10:08:06.809   POST api.tavily.com/search


[
    SearchResult(
        page_title='Staff Gen AI Engineer - Myworkdayjobs.com',
        page_description='',
        page_content='',
        url='https://jda.wd5.myworkdayjobs.com/en-US/jda_careers/job/Dallas/Principal-Data-Scientist_243429'
    ),
    SearchResult(
        page_title='Advisory National CT&I - AI Engineer - Manager',
        page_description='',
        page_content='Careers\n\n===============\n\n[Skip to main 
content](https://pwc.wd3.myworkdayjobs.com/US_Experienced_Careers/job/TX-Dallas/Advisory-National-CT-I---AI-Enginee
r---Manager_678688WD)\n\nCookies are small text files that are placed on your computer by the websites that you 
visit. They are widely used in order to make websites work, or work more efficiently, as well as to provide 
information to the owners of the site. The use of cookies is now standard for most websites. If you are 
uncomfortable with the use of cookies, you can manage and control them through your browser, including removing 
cookies by deleting them from your ‘browser history’ (cache) when you leave the site.\n\n[Cookie 
Policy](http://www.pwc.com/gx/en/site-information/cookie-information.html)\n\nRead Full Privacy 
Message\n\nDecline\n\nAccept Cookies\n\n[![Image 1: 
careers](https://pwc.wd3.myworkdayjobs.com/US_Experienced_Careers/assets/logo)](https://www.pwc.com/us/en/careers.h
tml)\n\nSign In\n\nCareers Homepage\n\n![Image 
2](https://pwc.wd3.myworkdayjobs.com/US_Experienced_Careers/assets/banner)\n\nAdvisory National CT&I - AI Engineer 
- Manager page is loaded\n\nAdvisory National CT&I - AI Engineer - 
Manager\n----------------------------------------------\n\n[Apply](https://pwc.wd3.myworkdayjobs.com/US_Experienced
_Careers/job/TX-Dallas/Advisory-National-CT-I---AI-Engineer---Manager_678688WD/apply)\n\nlocations TX-Dallas 
FL-Tampa GA-Atlanta IL-Chicago DC-Washington\n\nView All 6 Locations\n\ntime type Full time\n\nposted on Posted 
Yesterday\n\njob requisition id 678688WD\n\n**Industry/Sector**\n\nNot Applicable\n**Specialism**\n\nData 
Science\n**Management Level**\n\nManager\n**Job Description & Summary**\n\nAt PwC, our people in data and analytics
engineering focus on leveraging advanced technologies and techniques to design and develop robust data solutions 
for clients. They play a crucial role in transforming raw data into actionable insights, enabling informed 
decision-making and driving business growth.\n\nThose in data science and machine learning engineering at PwC will 
focus on leveraging advanced analytics and machine learning techniques to extract insights from large datasets and 
drive data-driven decision making. You will work on developing predictive models, conducting statistical analysis, 
and creating data visualisations to solve complex business problems.\nEnhancing your leadership style, you 
motivate, develop and inspire others to deliver quality. You are responsible for coaching, leveraging team member’s
unique strengths, and managing performance to deliver on client expectations. With your growing knowledge of how 
business works, you play an important role in identifying opportunities that contribute to the success of our Firm.
You are expected to lead with integrity and authenticity, articulating our purpose and values in a meaningful way. 
You embrace technology and innovation to enhance your delivery and encourage others to do the same.\n\nExamples of 
the skills, knowledge, and experiences you need to lead and deliver value at this level include but are not limited
to:\n\n*   Analyse and identify the linkages and interactions between the component parts of an entire system.\n*  
Take ownership of projects, ensuring their successful planning, budgeting, execution, and completion.\n*   Partner 
with team leadership to ensure collective ownership of quality, timelines, and deliverables.\n*   Develop skills 
outside your comfort zone, and encourage others to do the same.\n*   Effectively mentor others.\n*   Use the review
of work as an o

# Analyze Search Results


In [11]:
@dataclass
class SearchAnalyzerDeps:
    candidate_resume: str
    search_result: SearchResult


search_analyzer_agent = Agent[None, AnalyzedSearchResult](
    model,
    deps_type=SearchAnalyzerDeps,
    output_type=NativeOutput[AnalyzedSearchResult](
        AnalyzedSearchResult,
        name="AnalyzedSearchResult",
        description="Analysis search result determining if a job posting is a good fit for the candidate",
    ),
    name="Job Posting Analyzer",
)


@search_analyzer_agent.instructions
async def analyze_search_result_instructions(
    ctx: RunContext[SearchAnalyzerDeps],
) -> str:
    """Analyzes a job posting to determine if it is a good fit for the candidate."""
    deps = ctx.deps
    search_result = deps.search_result
    instructions = f"""
    Analyze this job posting and determine if the candidate is a good fit.

Job Posting:
Title: {search_result.page_title}
URL: {search_result.url}
Content: {search_result.page_content[:2000]}

Candidate Resume:
{ctx.deps.candidate_resume[:2000]}

Determine if this is:
1. A legitimate individual job posting (not a job board listing page or category page)
2. A good fit for the candidate based on their experience and skills


Set should_skip to true if this is not an individual job posting.
Set is_good_fit to true if the candidate is a good fit, false otherwise.
Provide clear reasoning for your decision.
Also set the job_title and job_summary fields to the job title and summary of the job posting.
"""
    return instructions


@logfire.instrument()
async def _analyze_search_result(
    search_result: SearchResult, candidate_resume: str
) -> AnalyzedSearchResult:
    result = await search_analyzer_agent.run(
        deps=SearchAnalyzerDeps(
            candidate_resume=candidate_resume,
            search_result=search_result,
        )
    )
    return result.output


@gb.step
async def analyze_search_result(
    ctx: StepContext[GraphState, GraphDeps, SearchResult],
) -> AnalyzedSearchResult:
    """Analyzes job postings to determine if they are a good fit for the candidate."""
    search_result = ctx.inputs
    candidate_resume = ctx.state.candidate_resume
    analyzed_search_result = await _analyze_search_result(
        search_result, candidate_resume
    )
    logfire.info(f"{analyzed_search_result=}")
    return analyzed_search_result


In [12]:
if run_tests:
    analyzed_search_result = await _analyze_search_result(
        search_results[0], candidate_resume
    )
    rich_print(analyzed_search_result)


10:08:07.941 Calling __main__._analyze_search_result
10:08:07.943   Job Posting Analyzer run
10:08:07.944     chat gpt-oss:20b
10:08:07.945       POST localhost/v1/chat/completions


AnalyzedSearchResult(
    page_title='Staff Gen AI Engineer - Myworkdayjobs.com',
    page_description='',
    page_content='',
    url='https://jda.wd5.myworkdayjobs.com/en-US/jda_careers/job/Dallas/Principal-Data-Scientist_243429',
    job_title='Staff Gen AI Engineer - Myworkdayjobs.com',
    job_summary='',
    should_skip=False,
    is_good_fit=False,
    fit_reasoning=None
)

# Analyze Prospective Job


In [13]:
@dataclass
class ProspectiveJobAnalyzerDeps:
    candidate_resume: str
    analyzed_search_result: AnalyzedSearchResult


prospective_job_analyzer_agent = Agent[None, ProspectiveJob](
    model,
    deps_type=ProspectiveJobAnalyzerDeps,
    output_type=NativeOutput[ProspectiveJob](
        ProspectiveJob,
        name="ProspectiveJob",
        description="Job Posting which includes a cover letter draft for the candidate.",
    ),
    name="Prospective Job Analyzer",
)


@prospective_job_analyzer_agent.instructions
async def analyze_prospective_job_instructions(
    ctx: RunContext[ProspectiveJobAnalyzerDeps],
) -> str:
    """Analyzes a job posting to determine if it is a good fit for the candidate."""
    deps = ctx.deps
    analyzed_search_result = deps.analyzed_search_result

    instructions = f"""
    Analyze this job posting and draft a cover letter for the candidate appropriate for their resume.
    Include a cover letter draft in the draft_cover_letter field along with a clear job_title and job_summary.
    Be sure to keep the URL unchanged in the url field.

Job Posting:
Title: {analyzed_search_result.job_title}
URL: {analyzed_search_result.url}
Job Description: {analyzed_search_result.page_content[:2000]}

Candidate Resume:
{ctx.deps.candidate_resume[:2000]}
"""
    return instructions


@logfire.instrument()
async def _analyze_prospective_job(
    candidate_resume: str, analyzed_search_result: AnalyzedSearchResult
) -> ProspectiveJob | None:
    if not analyzed_search_result.is_good_fit or analyzed_search_result.should_skip:
        return None
    result = await prospective_job_analyzer_agent.run(
        deps=ProspectiveJobAnalyzerDeps(
            candidate_resume=candidate_resume,
            analyzed_search_result=analyzed_search_result,
        )
    )
    prospective_job = result.output
    logfire.info(f"{prospective_job=}")
    return prospective_job


@gb.step
async def analyze_prospective_job(
    ctx: StepContext[GraphState, GraphDeps, AnalyzedSearchResult],
) -> ProspectiveJob | None:
    """Analyzes a job posting to determine if it is a good fit for the candidate."""
    state = ctx.state
    candidate_resume = state.candidate_resume
    analyzed_search_result = ctx.inputs
    prospective_job = await _analyze_prospective_job(
        candidate_resume, analyzed_search_result
    )
    if prospective_job is None:
        return None
    logfire.info(f"{prospective_job=}")
    if prospective_job.url not in [j.url for j in state.prospective_jobs]:
        state.prospective_jobs.append(prospective_job)
    return prospective_job

In [14]:
if run_tests:
    prospective_job = await _analyze_prospective_job(
        candidate_resume, analyzed_search_result
    )
    rich_print(prospective_job)

10:08:24.105 Calling __main__._analyze_prospective_job


None

# Construct the Agent Graph


In [15]:
from pydantic_graph.beta.join import reduce_list_append


collect = gb.join(reduce_list_append, initial_factory=list[ProspectiveJob])


@gb.step
@logfire.instrument()
async def output_decider(
    ctx: StepContext[GraphState, GraphDeps, None],
) -> Literal["continue_search"] | GraphOutput:
    state = ctx.state
    prospective_jobs = state.prospective_jobs
    if len(prospective_jobs) >= ctx.deps.target_num_prospective_jobs:
        return GraphOutput(prospective_jobs=prospective_jobs)
    elif state.search_iteration >= ctx.deps.max_search_iterations:
        return GraphOutput(prospective_jobs=prospective_jobs)
    else:
        return "continue_search"


gb.add(
    gb.edge_from(gb.start_node).to(search_prep),
    gb.edge_from(search_prep).to(search_job_postings),
    gb.edge_from(search_job_postings).map().to(analyze_search_result),
    gb.edge_from(analyze_search_result).to(analyze_prospective_job),
    gb.edge_from(analyze_prospective_job).to(collect),
    gb.edge_from(collect).to(output_decider),
    gb.edge_from(output_decider).to(
        gb.decision()
        .branch(gb.match(TypeExpression[Literal["continue_search"]]).to(search_prep))
        .branch(gb.match(TypeExpression[GraphOutput]).to(gb.end_node))
    ),
)

job_search_graph = gb.build()

In [16]:
from mermaid import Mermaid

mermaid_diagram = job_search_graph.render()
render = Mermaid(mermaid_diagram)
render

In [17]:
prospective_jobs = await job_search_graph.run(
    state=GraphState(
        candidate_resume=candidate_resume,
        search_query=initial_search_query,
    ),
    deps=GraphDeps(
        search_max_results=5,
        max_search_iterations=3,
        target_num_prospective_jobs=2,
    ),
)

10:08:24.794 run graph job_search_graph
10:08:24.795   run node search_prep
10:08:24.796     using initial search query executive leadership government defense jobs
10:08:24.796   run node search_job_postings
10:08:24.797     Calling __main__._search_job_postings
10:08:24.804       POST api.tavily.com/search
10:08:25.049   run node analyze_search_result
10:08:25.049     Calling __main__._analyze_search_result
10:08:25.051       Job Posting Analyzer run
10:08:25.051   run node analyze_search_result
10:08:25.051     Calling __main__._analyze_search_result
10:08:25.052       Job Posting Analyzer run
10:08:25.052   run node analyze_search_result
10:08:25.052     Calling __main__._analyze_search_result
10:08:25.053       Job Posting Analyzer run
10:08:25.053   run node analyze_search_result
10:08:25.054     Calling __main__._analyze_search_result
10:08:25.054       Job Posting Analyzer run
               run node analyze_search_result
                 Calling __main__._analyze_search_result

In [18]:
rich_print(prospective_jobs)

GraphOutput(
    prospective_jobs=[
        ProspectiveJob(
            job_title='Senior Executive Officer / Chief of Staff (Directors Action Group)',
            job_summary='Seeking a visionary, high‑impact senior executive to serve as Chief of Staff for Directors
Action Group, providing strategic counsel, operational oversight, and cross‑functional coordination to accelerate 
the organization’s mission and deliver results across government, defense, and institutional development 
initiatives.',
            url='https://careers-qinetiqus.icims.com/jobs/11651/senior-executive-officer-chief-of-staff-%28director
s-action-group%29/job',
            draft_cover_letter='[Your Name]\n[Street Address]\n[City, State ZIP]\n[Email Address] | [Phone 
Number]\n\n[Date]\n\nHiring Committee\nDirectors Action Group\n[Company Address]\n[City, State ZIP]\n\nDear Hiring 
Committee,\n\nI am writing to express my enthusiasm for the Senior Executive Officer / Chief of Staff position at 
Directors Action Group, as advertised. With a career spanning three centuries of founding institutions, commanding 
armies, and establishing the operational bedrock of a nation, I am well positioned to bring strategic clarity, 
disciplined execution, and visionary leadership to your organization’s ambitious portfolio.\n\n**Strategic 
Leadership & Institutional Design** – As the first President of the United States I designed and launched the 
Supreme Court, Congress, the Navy, and the First Bank – institutions that survive centuries of political 
turbulence. The same rigorous approach to constitutional architecture, fiscal stewardship, and governance 
structures will help Directors Action Group translate its bold policy objectives into sustainable, evidence‑based 
programs.\n\n**Logistics & Program Oversight** – During the Revolutionary War I oversaw supply chains that spanned 
the Atlantic and directed an army of 50,000+ in the logistics‑heavy theatres of Boston, New York, and Philadelphia.
This experience directly parallels the scale and complexity of your current initiatives in defense transformation 
and government‑wide operational efficiencies.\n\n**Diplomacy & Stakeholder Management** – Negotiating the Jay 
Treaty and the Treaty of San\u202fLorenzo required nuanced stakeholder analysis, risk assessment, and consensus 
building—skills that are essential for navigating the intersecting interests of legislators, contractors, and 
international partners that Directors Action Group engages.\n\n**Visionary Communication & Thought Leadership** – 
The Farewell Address remains one of the most enduring public texts in U.S. history, articulating a clear, 
forward‑looking national policy. I can leverage that legacy of persuasive messaging to amplify Directors Action 
Group’s narrative, secure bipartisan support, and galvanize action across a multi‑agency coalition.\n\n**Why me?** 
– While the scale of my past accomplishments may seem historic, the core competencies—strategic alignment, 
cross‑functional coordination, risk management, and people‑centric leadership—are timeless. I possess a unique 
blend of institutional memory and adaptability that is well suited to an organization that operates at the 
intersection of policy, defense, and high‑stakes advisory roles.\n\nI am excited about the opportunity to help 
Directors Action Group translate its powerful vision into decisive action and measurable impact. I look forward to 
discussing how my unique experience and strategic approach can strengthen your team’s collective 
effectiveness.\n\nThank you for your consideration.\n\nSincerely,\n\nGeorge Washington'
        )
    ]
)

# Non-Graph version


In [19]:
import asyncio


async def non_graph_job_search(
    candidate_resume: str,
    initial_search_query: str,
    search_max_results: int = 5,
    max_search_iterations: int = 1,
    target_num_prospective_jobs: int = 2,
):
    search_query = initial_search_query
    prospective_jobs = []
    search_iteration = 0
    done = False
    while not done:
        search_iteration += 1
        search_results = await _search_job_postings(search_query, search_max_results)
        analyzed_search_results = await asyncio.gather(
            *[_analyze_search_result(sr, candidate_resume) for sr in search_results]
        )
        maybe_prospective_jobs = await asyncio.gather(
            *[
                _analyze_prospective_job(candidate_resume, asr)
                for asr in analyzed_search_results
            ]
        )
        prospective_jobs.extend([p for p in maybe_prospective_jobs if p is not None])
        if len(prospective_jobs) >= target_num_prospective_jobs:
            done = True
        elif search_iteration >= max_search_iterations:
            done = True
        else:
            search_query = await _search_prep(search_query, search_results)
    return prospective_jobs


In [20]:
prospective_jobs = await non_graph_job_search(candidate_resume, initial_search_query)

10:15:32.245 Calling __main__._search_job_postings
10:15:32.255   POST api.tavily.com/search
10:15:32.512 Calling __main__._analyze_search_result
10:15:32.514   Job Posting Analyzer run
10:15:32.514 Calling __main__._analyze_search_result
10:15:32.515   Job Posting Analyzer run
10:15:32.515 Calling __main__._analyze_search_result
10:15:32.516   Job Posting Analyzer run
10:15:32.517 Calling __main__._analyze_search_result
10:15:32.517   Job Posting Analyzer run
             Calling __main__._analyze_search_result
               Job Posting Analyzer run
10:15:32.520     chat gpt-oss:20b
10:15:32.523       POST localhost/v1/chat/completions
             Calling __main__._analyze_search_result
               Job Posting Analyzer run
10:15:32.524     chat gpt-oss:20b
10:15:32.526       POST localhost/v1/chat/completions
             Calling __main__._analyze_search_result
               Job Posting Analyzer run
10:15:32.526     chat gpt-oss:20b
10:15:32.527       POST localhost/v1/chat/comp

In [21]:
rich_print(prospective_jobs)

[
    ProspectiveJob(
        job_title='Government Head – Department of Defense (DoD)',
        job_summary='Senior executive role responsible for leading the Department of Defense, setting strategic 
direction for national security, overseeing military operations, defense procurement and logistics, fostering 
inter‑agency collaboration, and ensuring the integrity and readiness of U.S. armed forces.',
        url='https://relx.wd3.myworkdayjobs.com/en-US/relx/job/Government-Head---DoD--Washington-DC-_R103946-1',
        draft_cover_letter='George Washington\n\nWashington, D.C.\n\nDate: ___________\n\nHiring Committee\nU.S. 
Department of Defense\nWashington, D.C.\n\nDear Hiring Committee,\n\nI am writing to formally express my interest 
in the Government Head position for the Department of Defense, as advertised on your careers portal. With a 
lifelong record of military command, strategic vision, and the establishment of enduring federal institutions, I am
uniquely positioned to lead today’s complex defense ecosystem and to advance America’s national security 
interests.\n\n**Military Leadership & Operational Excellence**\nI commanded the Continental Army from 1775‑1783, 
orchestrating pivotal victories at Trenton, Princeton, and Yorktown that secured American independence. My 
logistical acumen enabled the efficient movement of troops, supplies, and intelligence across hostile 
terrain—skills directly translatable to large‑scale defense logistics, supply chain resilience, and rapid 
deployment scenarios. Under my command, we established disciplined training protocols, standardized communication 
systems, and integrated field medicine practices—early examples of what today’s joint force readiness programs rely
upon.\n\n**Strategic Policy & Institutional Foundations**\nAs the first President of the United States, I founded 
key defense‑related institutions—including the Navy, the U.S. Mint, and the First Bank of the United States—while 
structuring long‑term national fiscal policy that underpins defense budgeting. I introduced the Judiciary Act and 
the Naturalization Act, forging a legal framework that continues to govern the intersection of law, commerce, and 
defense. My experience negotiating the Jay Treaty and the Treaty of San\u202fLorenzo showcases my ability to 
maintain national security through diplomatic channels, a critical component of today’s defense partnership 
ecosystem.\n\n**Fiscal Stewardship & Resource Management**\nI designed and executed large‑scale financial 
programs—including the first U.S. federal banking system and the financing of the Revolutionary War—which required 
meticulous budgeting, risk assessment, and oversight of multibillion‑dollar expenditures. These competencies 
directly align with the DoD’s need to manage complex, multi‑year procurement contracts, sustain R&D pipelines, and 
optimize defense spending to protect American interests.\n\n**Visionary Leadership & National Unity**\nThrough the 
Proclamation of Neutrality, the suppression of the Whiskey Rebellion, and my Farewell Address, I proved adept at 
balancing domestic stability with foreign‑policy restraint, preserving unity during periods of crisis. I would 
bring this steady, principled leadership to the Department, ensuring that defense policy remains anchored to 
American values while navigating the multifaceted threats of the 21st century.\n\nI am eager to bring my experience
in strategic command, institution-building, fiscal stewardship, and diplomatic negotiation to the Deputy 
Secretary’s office. I believe that my proven record of turning visionary ideas into enduring national structures 
will enable me to guide the Department of Defense toward unprecedented levels of innovation, readiness, and global 
partnership.\n\nThank you for considering my application. I look forward to the opportunity to discuss how my 
background, skills, and vision align with the Department’s mission and strategic 
objectives.\n\nRespectfully,\n\nG